# Tratamento inicial de dados

Aqui iniciaremos nosso tratemento inicial de dados, vamos converter o JSON com as matérias coletadas para um Dataframe e incluir as colunas com os resumos de cada matéria para cada persona.

Estrutura do dataframe:
- id: Identificador associado a cada matéria
- categoria: Categoria a qual a matéria pertence
- titulo: Titulo da matéria
- subtitulo: Subtítulo da matéria
- texto: Texto completo da matéria
- resumo_persona_1: Resumo gerado usando a LLM escolhida para a persona 1
- resumo_persona_2: Resumo gerado usando a LLM escolhida para a persona 2
- resumo_persona_3: Resumo gerado usando a LLM escolhida para a persona 3

### Importando bibliotecas utilizadas

In [1]:
import json
import pandas as pd
import warnings
from summary_builder import SummaryBuilder

warnings.filterwarnings('ignore')


### Leitura dos dados

In [2]:
with open("./materias.json", "r") as f:
    materias = json.load(f)

with open("./personas.json", "r") as f:
    personas = json.load(f)

# print(json.dumps(materias, indent=2))
# print(json.dumps(personas, indent=2))

In [3]:
df = pd.DataFrame(materias["materias"], columns=["id", "categoria", "url", "titulo", "subtitulo", "texto", "resumo_persona_1", "resumo_persona_2", "resumo_persona_3"])

df.set_index("id", inplace=True)

# df["resumo_persona_1"].fillna("", inplace=True)
# df["resumo_persona_2"].fillna("", inplace=True)
df["resumo_persona_3"].fillna("", inplace=True)

df

,categoria,url,titulo,subtitulo,texto,resumo_persona_1,resumo_persona_2,resumo_persona_3
id,,,,,,,,
10,esporte,https://ge.globo.com/futebol/times/fluminense/...,"Prestes a pegar o Borussia, Thiago Silva, do F...",Capitão tricolor passou 16 anos no futebol eur...,Thiago Silva compara diferenças do futebol eur...,NaN,NaN,


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 10 to 10
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   categoria         1 non-null      object
 1   url               1 non-null      object
 2   titulo            1 non-null      object
 3   subtitulo         1 non-null      object
 4   texto             1 non-null      object
 5   resumo_persona_1  1 non-null      object
 6   resumo_persona_2  1 non-null      object
 7   resumo_persona_3  1 non-null      object
dtypes: object(8)
memory usage: 72.0+ bytes


### Gerando texto para cada persona

In [ ]:
summary_builder = SummaryBuilder()
resumos = {
    # "persona_1": [],
    # "persona_2": [],
    "persona_3": []
}

for index, row in df.iterrows():
    print(f"Gerando resumos do texto {index} - Título: {row["titulo"]}")
    texto = row["texto"]
    texto_categoria = row["categoria"]

    for persona in personas["personas"]:
        if persona["id"] != 3:
            continue
        print(f"Gerando texto para: {persona['nome']}")

        _id = persona["id"]
        preferencias = persona["scores"]
        time_do_coracao = persona["time_do_coracao"]
        nome_persona = persona['nome']
        
        resumo = summary_builder.get(nome_persona, texto, texto_categoria, preferencias, time_do_coracao)
        resumos[f"persona_{_id}"].append(resumo)
        
    
    print("\n")
        

Gerando resumos do texto 10 - Título: Prestes a pegar o Borussia, Thiago Silva, do Fluminense, cita maior diferença entre Europa e Brasil
Gerando texto para: renata
{'tecnologia': 0.2, 'economia': 0.9, 'ciencia': 0.25, 'saude': 0.5, 'educacao': 0.45, 'turismo': 0.1, 'politica': 0.95, 'esporte': 0.15}

            Você é um assistente especialista em criar resumos de notícias personalizados com base no interesse do usuário.

            Receba:
            1. O conteúdo completo de uma notícia.
            2. Um JSON com as preferências completas do usuário (valores de 0 a 1).
            3. Um time de futebol (se houver), relevante apenas se o tema for "esporte".

            Tarefa:
            - Faça um resumo da notícia, em português.

            - Sabendo que o usuário que receberá este resumo ama esporte, mas a notícia é sobre seu time rival, não dê tanto foco aos detalhes da notícia. Pode usar um tom irônico, se possível.
            - Seja sucinto no resumo, entregando a princi

### Salvando resumos gerados no Dataframe

In [7]:
for i in range(len(personas["personas"])):
    if (i+1) != 2:
        continue
    df[f"resumo_persona_{i + 1}"] = resumos[f"persona_{i + 1}"]
    

In [7]:
df

,categoria,url,titulo,subtitulo,texto,resumo_persona_1,resumo_persona_2,resumo_persona_3
id,,,,,,,,
10,esporte,https://ge.globo.com/futebol/times/fluminense/...,"Prestes a pegar o Borussia, Thiago Silva, do F...",Capitão tricolor passou 16 anos no futebol eur...,Thiago Silva compara diferenças do futebol eur...,NaN,NaN,"Thiago Silva, zagueiro do Fluminense e com vas..."


In [8]:
df.loc[10, "resumo_persona_2"]

'Thiago Silva resolveu dar uma aula sobre as diferenças entre o futebol europeu e o brasileiro, destacando a intensidade do jogo. Ele cita o clima como grande responsável pela oscilação de desempenho na terra das chuteiras canhotas. No entanto, o zagueiro do Fluminense parece estar pronto para encarar o Borussia Dortmund na Copa do Mundo de Clubes, tentando levar a intensidade europeia para o calorzinho do Brasil. E enquanto isso, o Flamengo segue assistindo de longe a "superação" do rival!'

### Exportando para .csv

In [ ]:
df.to_csv("resumos.csv", sep=";", encoding="utf-8")